### Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('../data/data.csv')
df.dropna(inplace=True)
df.head(5)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


### Data Preprocessing

In [ ]:
# df['month'] = pd.to_datetime(df['month'], format='%Y-%m')
# df.head(5)

In [4]:
processed_df = df[['month', 'town', 'resale_price']]
processed_df

,month,town,resale_price
0,2017-01,ANG MO KIO,232000.0
1,2017-01,ANG MO KIO,250000.0
2,2017-01,ANG MO KIO,262000.0
3,2017-01,ANG MO KIO,265000.0
4,2017-01,ANG MO KIO,265000.0
...,...,...,...
173329,2024-02,YISHUN,820000.0
173330,2024-02,YISHUN,850000.0
173331,2024-02,YISHUN,795000.0
173332,2024-02,YISHUN,935000.0


In [5]:
# group data by month and town, and retrieve the median resale price for each month for each town
processed_df = processed_df.groupby(['month', 'town']).median().reset_index()
processed_df

,month,town,resale_price
0,2017-01,ANG MO KIO,336500.0
1,2017-01,BEDOK,359000.0
2,2017-01,BISHAN,604000.0
3,2017-01,BUKIT BATOK,338000.0
4,2017-01,BUKIT MERAH,540000.0
...,...,...,...
2229,2024-02,SERANGOON,651000.0
2230,2024-02,TAMPINES,630000.0
2231,2024-02,TOA PAYOH,534000.0
2232,2024-02,WOODLANDS,560000.0


In [6]:
processed_df['month'] = pd.to_datetime(processed_df['month'], format='%Y-%m')
processed_df

,month,town,resale_price
0,2017-01-01,ANG MO KIO,336500.0
1,2017-01-01,BEDOK,359000.0
2,2017-01-01,BISHAN,604000.0
3,2017-01-01,BUKIT BATOK,338000.0
4,2017-01-01,BUKIT MERAH,540000.0
...,...,...,...
2229,2024-02-01,SERANGOON,651000.0
2230,2024-02-01,TAMPINES,630000.0
2231,2024-02-01,TOA PAYOH,534000.0
2232,2024-02-01,WOODLANDS,560000.0


In [7]:
# get unique towns
towns = processed_df['town'].unique()
towns

array(['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH',
       'BUKIT PANJANG', 'BUKIT TIMAH', 'CENTRAL AREA', 'CHOA CHU KANG',
       'CLEMENTI', 'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST',
       'KALLANG/WHAMPOA', 'MARINE PARADE', 'PASIR RIS', 'PUNGGOL',
       'QUEENSTOWN', 'SEMBAWANG', 'SENGKANG', 'SERANGOON', 'TAMPINES',
       'TOA PAYOH', 'WOODLANDS', 'YISHUN'], dtype=object)

In [12]:
train_df = processed_df.copy()

In [37]:
# for multiple towns

forecast_dfs = []

for town in towns:
    train_df_town = train_df[train_df['town'] == town]
    # train_df_town.set_index('month', inplace=True)
    # order = (2, 2, 2)

    # model = ARIMA(train_df_town['resale_price'], order=order)
    # results = model.fit()
    # forecast = results.forecast(steps=84)
    
    # # Convert forecast series to DataFrame
    # forecast_df = forecast.to_frame()
    # forecast_df.reset_index(inplace=True)
    # forecast_df.columns = ['month', 'resale_price']
    # forecast_df['town'] = town
    train_df_town.set_index('month', inplace=True)
    model = ARIMA(train_df_town['resale_price'], order=(1,2,1))
    # MODEL - USE THIS
    results = model.fit()
    x=results.forecast(84)

    # Generate a sequence of future months based on the last known month in the training data
    last_month = train_df_town.index[-1]
    future_months = [last_month + DateOffset(months=i) for i in range(1, 85)]

    # Convert the forecasted values to a DataFrame
    forecast_df = x.to_frame()
    forecast_df.reset_index(drop=True, inplace=True)  # Reset index to start from 0
    forecast_df.columns = ['resale_price']  # Update column name

    # Add the 'month' and 'town' columns
    forecast_df['month'] = future_months
    forecast_df['town'] = town

    # Reorder columns
    forecast_df = forecast_df[['month', 'resale_price', 'town']]
    
    # Append forecast DataFrame to the list
    forecast_dfs.append(forecast_df)

# Concatenate all forecast DataFrames
forecast_df_all = pd.concat(forecast_dfs, ignore_index=True)

# Output the concatenated forecast DataFrame
forecast_df_all

,month,resale_price,town
0,2024-03-01,480380.531625,ANG MO KIO
1,2024-04-01,485816.405843,ANG MO KIO
2,2024-05-01,491260.328808,ANG MO KIO
3,2024-06-01,496705.422343,ANG MO KIO
4,2024-07-01,502150.686118,ANG MO KIO
...,...,...,...
2179,2030-10-01,853391.851400,YISHUN
2180,2030-11-01,858479.108850,YISHUN
2181,2030-12-01,863566.366301,YISHUN
2182,2031-01-01,868653.623751,YISHUN


In [38]:
import json
import pandas as pd

# Assuming forecast_df_all is your DataFrame containing the data

# Convert the 'month' column to datetime
forecast_df_all['month'] = pd.to_datetime(forecast_df_all['month'])

# Group the data by town and year
grouped_data = forecast_df_all.groupby(['town', forecast_df_all['month'].dt.year])

# Initialize a dictionary to store the results
result_dict = {}

# Iterate over the grouped data
for (town, year), group in grouped_data:
    # Calculate the median resale price for each group
    median_price = int(group['resale_price'].median())
    
    # If the town is not in the result_dict, add it
    if town not in result_dict:
        result_dict[town] = {
            "town": str(town),
            "data": []
        }
    
    # Append the data for this group to the result list
    result_dict[town]["data"].append({
        "medianPrice": median_price, "year": int(year)
    })

# Convert the dictionary values to a list
result = list(result_dict.values())

# Convert the result to JSON
json_result = json.dumps(result, indent=2)
print(json_result)


[
  {
    "town": "ANG MO KIO",
    "data": [
      {
        "medianPrice": 504873,
        "year": 2024
      },
      {
        "medianPrice": 564771,
        "year": 2025
      },
      {
        "medianPrice": 630115,
        "year": 2026
      },
      {
        "medianPrice": 695458,
        "year": 2027
      },
      {
        "medianPrice": 760802,
        "year": 2028
      },
      {
        "medianPrice": 826145,
        "year": 2029
      },
      {
        "medianPrice": 891489,
        "year": 2030
      },
      {
        "medianPrice": 929606,
        "year": 2031
      }
    ]
  },
  {
    "town": "BEDOK",
    "data": [
      {
        "medianPrice": 467873,
        "year": 2024
      },
      {
        "medianPrice": 504450,
        "year": 2025
      },
      {
        "medianPrice": 544354,
        "year": 2026
      },
      {
        "medianPrice": 584257,
        "year": 2027
      },
      {
        "medianPrice": 624161,
        "year": 2028
      },
      {


In [39]:
# save json_result to a file
with open('forecast.json', 'w') as f:
    f.write(json_result)